# Customer Journey

What to do?
optimize customer journeys which maximize their satisfaction and lifetime value. predict customer churn and interpret its drivers, measure, and forecast customer lifetime value, and finally, build customer segments based on their product purchase patterns

Objective:
    use customer data from a telecom company to predict churn, construct a recency-frequency-monetary dataset from an online retailer for customer lifetime value prediction, and build customer segments from product purchase data from a grocery shop.

Questions we want to answer using machine learning:
    1. Which customers will churn? (classification)
        : collect data on purchase patterns prior to churning
    2. Which customers will buy again? (classification)
    3. How much will customers spend in the next 30 days? (regression)
    

## Preliminary Analysis

explore the dataset and now know key elements of its structure.

In [ ]:
import pandas as pd
import sklearn

In [52]:
import pandas as pd
telco_raw = pd.read_csv('/Users/jihunlee/Desktop/telco.csv')

In [53]:
telco_raw.head(5)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [2]:
# Print the data types of telco_raw dataset
print(telco_raw.dtypes)

# Print the header of telco_raw dataset
print(telco_raw.head())

# Print the number of unique values in each telco_raw column
print(telco_raw.nunique())

customerID           object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object
   customerID  gender  SeniorCitizen Partner Dependents  tenure PhoneService  \
0  7590-VHVEG  Female              0     Yes         No       1           No   
1  5575-GNVDE    Male              0      No         No      34          Yes   
2  3668-QPYBK    Male              0      No         No       2          Yes   
3  7795-CFOCW    Male              0      No         No      45           No  

separate the categorical and numerical columns and are ready to transform them

In [54]:
# Store customerID and Churn column names
custid = ['customerID']
target = ['Churn']

# Store categorical column names names that have less than 5 unique values.
categorical = telco_raw.nunique()[telco_raw.nunique() < 5].keys().tolist()

# Remove target from the list of categorical variables
categorical.remove(target[0])

# Store numerical column names
numerical = [x for x in telco_raw.columns if x not in custid + target + categorical]

perform one-hot encoding on the categorical variables and then scale the numerical columns

In [55]:
telco_raw['TotalCharges'] = pd.to_numeric(telco_raw['TotalCharges'], errors='coerce')

In [5]:
# Perform one-hot encoding to categorical variables 
telco_raw = pd.get_dummies(data = telco_raw, columns = categorical, drop_first=True)

# Initialize StandardScaler instance
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# Fit and transform the scaler on numerical columns
scaled_numerical = scaler.fit_transform(telco_raw[numerical])

# Build a DataFrame from scaled_numerical
scaled_numerical = pd.DataFrame(scaled_numerical, columns=numerical)

In [13]:
telco_raw = telco_raw.drop(['tenure', 'MonthlyCharges', 'TotalCharges'], axis=1)

In [16]:
# concatenate by columns
df = pd.concat([telco_raw, scaled_numerical], axis=1)

build an end-to-end machine learning model

In [27]:
# drop NaN
df = df.dropna()

In [28]:
X = df.iloc[:,:-1]
Y = df.iloc[:,-1]

In [29]:
# Split X and Y into training and testing datasets
from sklearn.model_selection import train_test_split
train_X, test_X, train_Y, test_Y = train_test_split(X, Y, test_size=0.25)

# Ensure training dataset has only 75% of original X data
print(train_X.shape[0] / X.shape[0])

# Ensure testing dataset has only 25% of original X data
print(test_X.shape[0] / X.shape[0])

0.75
0.25


The decision tree is a list of machine-learned if-else rules that decide in the telecom churn case, whether customers will churn or not

In [31]:
# Initialize the model with max_depth set at 5
from sklearn import tree
mytree = tree.DecisionTreeClassifier(max_depth = 5)

# Fit the model on the training data
treemodel = mytree.fit(train_X, train_Y)

# Predict values on the testing data
pred_Y = treemodel.predict(test_X)

# Measure model performance on testing data
from sklearn.metrics import accuracy_score
accuracy_score
accuracy_score(pred_Y, test_Y)

0.7940841865756542

build a more complex decision tree with additional parameters to predict customer churn.

In [33]:
# Initialize the Decision Tree
clf = tree.DecisionTreeClassifier(max_depth = 7, 
               criterion = 'gini', 
               splitter  = 'best')

# Fit the model to the training data
clf = clf.fit(train_X, train_Y)

# Predict the values on test dataset
pred_Y = clf.predict(test_X)

# Print accuracy values
import numpy as np
print("Training accuracy: ", np.round(clf.score(train_X, train_Y), 3)) 
print("Test accuracy: ", np.round(accuracy_score(test_Y, pred_Y), 3))

Training accuracy:  0.824
Test accuracy:  0.782


## Churn Prediction Models

What is churn?
- Churn happens when a customer stops buying / engaging
- The business context could be contractual or non-contractual
- Sometimes churn can be viewed as either voluntary or involuntary

Types of churn
1. Contractual (phone subscription, TV streaming subscription)
2. Non-contractual (grocery shopping, online shopping)

Modeling different types of churn
- Non-contractual churn is harder to dene and model, as there's no explicit customer decision

explore the churn distribution and split the data into training and testing before you proceed to modeling

In [36]:
# Print the unique Churn values
print(set(df['Churn']))

# Calculate the ratio size of each churn group
df.groupby(['Churn']).size() / df.shape[0] * 100

# Import the function for splitting data to train and test
from sklearn.model_selection import train_test_split

# Split the data into train and test
train, test = train_test_split(df, test_size = .25)

{'Yes', 'No'}


separate the features and target variables into different datasets

In [37]:
# Store column names from `telcom` excluding target variable and customer ID
cols = [col for col in df.columns if col not in custid + target]

# Extract training features
train_X = train[cols]

# Extract training target
train_Y = train[target]

# Extract testing features
test_X = test[cols]

# Extract testing target
test_Y = test[target]

### Logistic Regression

fit a logistic regression on the training part of the telecom churn dataset, and then predict labels on the unseen test set.

In [41]:
# Fit logistic regression on training data
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
logreg.fit(train_X, train_Y)

# Predict churn labels on testing data
pred_test_Y = logreg.predict(test_X)

# Calculate accuracy score on testing data
test_accuracy = accuracy_score(test_Y, pred_test_Y)

# Print test accuracy score rounded to 4 decimals
print('Test accuracy:', round(test_accuracy, 4))

Test accuracy: 0.8134


/Users/jihunlee/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/jihunlee/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


 run a logistic regression model on scaled data with L1 regularization to perform feature selection alongside model building

In [42]:
# Initialize logistic regression instance 
logreg = LogisticRegression(penalty='l1', C=0.025, solver='liblinear')

# Fit the model on training data
logreg.fit(train_X, train_Y)

# Predict churn values on test data
pred_test_Y = logreg.predict(test_X)

# Print the accuracy score on test data
print('Test accuracy:', round(accuracy_score(test_Y, pred_test_Y), 4))

Test accuracy: 0.8055


/Users/jihunlee/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


tune the C parameter for the L1 regularization to discover the one which reduces model complexity while still maintaining good model performance metrics

A list C has been created with the possible values. The l1_metrics array has been built with 3 columns, with the first being the C values, and the next two being placeholders for non-zero coefficient counts and the recall score of the model. 

In [ ]:
GridSearchCV 

In [50]:
# Run a for loop over the range of C list length
from sklearn.metrics import recall_score 
C = np.linspace(0.1,1,10)
l1_metrics = np.empty([len(C), 3])
for index in range(0, len(C)):
  # Initialize and fit Logistic Regression with the C candidate
  logreg = LogisticRegression(penalty='l1', C=C[index], solver='liblinear')
  logreg.fit(train_X, train_Y)
  # Predict churn on the testing data
  pred_test_Y = logreg.predict(test_X)
  # Create non-zero count and recall score columns
  l1_metrics[index,1] = np.count_nonzero(logreg.coef_)
  l1_metrics[index,2] = recall_score(test_Y, pred_test_Y, pos_label='Yes')

# Name the columns and print the array as pandas DataFrame
col_names = ['C','Non-Zero Coeffs','Recall']
print(pd.DataFrame(l1_metrics, columns=col_names))

/Users/jihunlee/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/jihunlee/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/jihunlee/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/jihunlee/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array w

               C  Non-Zero Coeffs    Recall
0  2.237494e-314             18.0  0.519651
1  2.237494e-314             22.0  0.532751
2  2.237494e-314             24.0  0.539301
3  2.237494e-314             24.0  0.539301
4  2.237494e-314             20.0  0.541485
5  2.237494e-314             27.0  0.541485
6  2.237494e-314             27.0  0.541485
7  2.237494e-314             27.0  0.541485
8  2.237494e-314             25.0  0.541485
9  2.237497e-314             27.0  0.541485


/Users/jihunlee/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Explore the coefficients of the logistic regression to understand what is driving churn to go up or down. Extract the logistic regression coefficients from your fitted model, and calculate their exponent to make them more interpretable. extracted and transformed logistic regression coefficients and can now understand which of them increase of decrease the odds of churn

In [51]:
# Combine feature names and coefficients into pandas DataFrame
feature_names = pd.DataFrame(train_X.columns, columns = ['Feature'])
log_coef = pd.DataFrame(np.transpose(logreg.coef_), columns = ['Coefficient'])
coefficients = pd.concat([feature_names, log_coef], axis = 1)

# Calculate exponent of the logistic regression coefficients
coefficients['Exp_Coefficient'] = np.exp(coefficients['Coefficient'])

# Remove coefficients that are equal to zero
coefficients = coefficients[coefficients['Coefficient']!=0]

# Print the values sorted by the exponent coefficient
print(coefficients.sort_values(by=['Exp_Coefficient']))

                                  Feature  Coefficient  Exp_Coefficient
27                                 tenure    -1.420911         0.241494
22                      Contract_Two year    -1.409622         0.244236
4                        PhoneService_Yes    -0.892115         0.409788
21                      Contract_One year    -0.564353         0.568728
10                     OnlineSecurity_Yes    -0.458129         0.632466
16                        TechSupport_Yes    -0.447104         0.639477
9      OnlineSecurity_No internet service    -0.381723         0.682684
12                       OnlineBackup_Yes    -0.229143         0.795214
3                          Dependents_Yes    -0.143077         0.866687
14                   DeviceProtection_Yes    -0.069104         0.933230
24  PaymentMethod_Credit card (automatic)    -0.050386         0.950862
8                      InternetService_No    -0.010164         0.989888
11       OnlineBackup_No internet service    -0.004534         0

### Decision Tree 

fit a decision tree on the training set of the telecom dataset, and then predict labels on the unseen testing data, and calculate the accuracy of your model predictions

In [ ]:
# Initialize decision tree classifier
mytree = tree.DecisionTreeClassifier()

# Fit the decision tree on training data
mytree.fit(train_X, train_Y)

# Predict churn labels on testing data
pred_test_Y = mytree.predict(test_X)

# Calculate accuracy score on testing data
test_accuracy = accuracy_score(test_Y, pred_test_Y)

# Print test accuracy
print('Test accuracy:', round(test_accuracy, 4))

tune the max_depth parameter of the decision tree to discover the one which reduces over-fitting while still maintaining good model performance metrics

In [ ]:
# Run a for loop over the range of depth list length
for index in range(0, len(depth_list)):
  # Initialize and fit decision tree with the `max_depth` candidate
  mytree = DecisionTreeClassifier(max_depth=depth_list[index])
  mytree.fit(train_X, train_Y)
  # Predict churn on the testing data
  pred_test_Y = mytree.predict(test_X)
  # Calculate the recall score 
  depth_tuning[index,1] = recall_score(test_Y, pred_test_Y)

# Name the columns and print the array as pandas DataFrame
col_names = ['Max_Depth','Recall']
print(pd.DataFrame(depth_tuning, columns=col_names))

extract the if-else rules from the decision tree and plot them to identify the main drivers of the churn

In [ ]:
# Export graphviz object from the trained decision tree 
exported = tree.export_graphviz(decision_tree=mytree, 
                                # Assign feature names
                                out_file=None, feature_names=train_X.columns, 
                                # Set precision to 1 and add class names
                                precision=1, class_names=['Not churn','Churn'], filled = True)

# Call the Source function and pass the exported graphviz object
graph = graphviz.Source(exported)

# Display the decision tree
display_image("/User/jihunlee/Desktop/decision_tree_rules.png")

# CLV Prediction

What is CLV?
- Measurement of customer value
- Can be historical or predicted
- Multiple approaches, depends on business type
- Some methods are formula-based, some are predictive and distribution based

Calculating Historical CLV
- Sum revenue of all past transactions
- Multiply by the profit margin
- Alternatively - sum profit of all past transactions, if available

Problems are:
- does not account for tenure, retention and churn
- does not account for new customers and their future revenue

Basic CLV
- Multiply average revenue with profit margin to get average profit
- Multiply it with average customer lifespan
CLV = Avg. Revenue * Profit Margin * Avg. Lifespan

Granular CLV
- Multiply average revenue per purchase with average frequency and with prot margin
- Multiply it with average customer lifespan
- Accounts for both average revenue per transaction and average frequency per period
CLV = Avg. revenue per purchase * Avg. Frequency * Profit Margin * Avg. Lifespan

Traditional CLV Formula
- Multiply average revenue with profit margin
- Multiple average prot with the retention to churn rate
- Churn can be derived from retention and equals 1 minus retention rate
- Accounts for customer loyalty, most popular approach
CLV = Avg. Revenue * Profit Margin * Retention Rate/ Churn Rate

### Preparing Cohort Dataset

use the monthly cohort activity dataset to calculate retention and churn values

In [ ]:
# Extract cohort sizes from the first column of cohort_counts
cohort_sizes = cohort_counts.iloc[:,0]

# Calculate retention by dividing the counts with the cohort sizes
retention = cohort_counts.divide(cohort_sizes, axis=0)

# Calculate churn
churn = 1 - retention

# Print the retention table
print(retention)

calculate the overall mean retention and churn rates. 

use the .mean() method twice in a row (this is called "chaining") to calculate the overall mean. exclude the first month values (first column) from this calculation as they are constant given this is the first month the customers have been active therefore their retention will be 100% and churn will be 0% for all cohorts.

In [ ]:
# Calculate the mean retention rate
retention_rate = retention.iloc[:,1:].mean().mean()

# Calculate the mean churn rate
churn_rate = churn.iloc[:,1:].mean().mean()

# Print rounded retention and churn rates
print('Retention rate: {:.2f}; Churn rate: {:.2f}'.format(retention_rate, churn_rate))

calculate the basic CLV which multiplies average monthly spent with the projected customer lifespan

In [ ]:
# Calculate monthly spend per customer
monthly_revenue = online.groupby(['CustomerID','InvoiceMonth'])['TotalSum'].sum().mean()

# Calculate average monthly spend
monthly_revenue = np.mean(monthly_revenue)

# Define lifespan to 36 months
lifespan_months = 36

# Calculate basic CLV
clv_basic = monthly_revenue * lifespan_months

# Print the basic CLV value
print('Average basic CLV is {:.1f} USD'.format(clv_basic))

more granular data and can give a better customer lifetime value estimate

In [ ]:
# Calculate average revenue per invoice
revenue_per_purchase = online.groupby(['InvoiceNo'])['TotalSum'].mean().mean()

# Calculate average number of unique invoices per customer per month
frequency_per_month = online.groupby(['CustomerID','InvoiceMonth'])['InvoiceNo'].nunique().mean()

# Define lifespan to 36 months
lifespan_months = 36

# Calculate granular CLV
clv_granular = revenue_per_purchase * frequency_per_month * lifespan_months

Calculate traditional CLV: most popular descriptive CLV models that accounts for the retention and churn rates

In [ ]:
# Calculate monthly spend per customer
monthly_revenue = online.groupby(['CustomerID','InvoiceMonth'])['TotalSum'].sum().mean()

# Calculate average monthly retention rate
retention_rate = retention.iloc[:,1:].mean().mean()

# Calculate average monthly churn rate
churn_rate = 1 - retention_rate

# Calculate traditional CLV 
clv_traditional = monthly_revenue * (retention_rate / churn_rate)

# Print traditional CLV and the retention rate values
print('Average traditional CLV is {:.1f} USD at {:.1f} % retention_rate'.format(clv_traditional, retention_rate*100))

# CLV Regression on RFM Dataset

build recency, frequency, monetary value and other customer level features.  These features capture highly predictive customer behavior patterns

In [ ]:
# Define the snapshot date
NOW = dt.datetime(2011,11,1)

# Calculate recency by subtracting current date from the latest InvoiceDate
features = online_X.groupby('CustomerID').agg({
  'InvoiceDate': lambda x: (NOW - x.max()).days,
  # Calculate frequency by counting unique number of invoices
  'InvoiceNo': pd.Series.nunique,
  # Calculate monetary value by summing all spend values
  'TotalSum': np.sum,
  # Calculate average and total quantity
  'Quantity': ['mean', 'sum']}).reset_index()

# Rename the columns
features.columns = ['CustomerID', 'recency', 'frequency', 'monetary', 'quantity_avg', 'quantity_total']a

build a pandas pivot table with customers as rows, invoice months as columns, and number of invoice counts as values

In [ ]:
# Build a pivot table counting invoices for each customer monthly
cust_month_tx = pd.pivot_table(data=online, values='InvoiceNo',
                               index=['CustomerID'], columns=['InvoiceMonth'],
                               aggfunc=pd.Series.nunique, fill_value=0)

# Store November 2011 data column name as a list
target = ['2011-11']

# Store target value as `Y`
Y = cust_month_tx[target]

identify the names of the target variable and the feature columns, extract the data, and split them into training and testing

In [ ]:
# Store customer identifier column name as a list
custid = ['CustomerID']

# Select feature column names excluding customer identifier
cols = [col for col in features.columns if col not in custid]

# Extract the features as `X`
X = features[cols]

# Split data to training and testing
trainX, test_X, train_Y, test_Y = train_test_split(X, Y, test_size=0.25, random_state=99)

# Regression Models

### Performance Metrics

- Root mean squared error (RMSE) - Square root ofthe average squared difference between prediction and actuals
- Mean absolute error (MAE) - Average absolute difference between prediction and actuals
- Mean absolute percentage error (MAPE) - Average percentage difference between prediction and actuals (actuals can't be zeros)

build the model on the training data, and predict values on testing data

In [ ]:
# Initialize linear regression instance
linreg = LinearRegression()

# Fit the model to training dataset
linreg.fit(train_X, train_Y)

# Predict the target variable for training data
train_pred_Y = linreg.predict(train_X)

# Predict the target variable for testing data
test_pred_Y = linreg.predict(test_X)

measure the regression performance on both training and testing data with two metrics - root mean squared error and mean absolute error. measure how "close" are the model predictions compared to actual values.

In [ ]:
# Calculate root mean squared error on training data
rmse_train = np.sqrt(mean_squared_error(train_Y, train_pred_Y))

# Calculate mean absolute error on training data
mae_train = mean_absolute_error(train_Y, train_pred_Y)

# Calculate root mean squared error on testing data
rmse_test = np.sqrt(mean_squared_error(test_Y, test_pred_Y))

# Calculate mean absolute error on testing data
mae_test = mean_absolute_error(test_Y, test_pred_Y)

# Print the performance metrics
print('RMSE train: {}; RMSE test: {}\nMAE train: {}, MAE test: {}'.format(rmse_train, rmse_test, mae_train, mae_test))

not all model coefficients are statistically significant and look at the model summary table to explore their significance. The statsmodels library provides this functionality. Print the model summary table, explore which variables have the p-value lower than 0.05 (i.e. lower than 5%) to make sure the coefficient is significant

In [ ]:
# Import `statsmodels.api` module
import statsmodels.api as sm

# Initialize model instance on the training data
olsreg = sm.OLS(train_Y, train_X)

# Fit the model
olsreg = olsreg.fit()

# Print model summary
print(olsreg.summary())